## Capstone Project - Diamond Prices (Part 1 - Round Only)

In [ ]:
#I was able to get a large dataset of diamond prices from recent years (2017+) through a contact at a diamond aggregator company
#It has more features than the public datasets I looked at on Kaggle
#In addition, it has a robust amount of data for fancy shapes (non-round), while the Kaggle dataset was only for round diamonds
#I will split my model into round and non-round to reflect industry pricing standards (Rapaport guide)
#Example of Rapaport pricing guide: https://www.diamonds.net/Prices/RapaportPriceGuide.aspx

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.datasets import fetch_openml
from sklearn import svm
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.ensemble import RandomForestRegressor #added this one to try
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, Normalizer
from xgboost import XGBRegressor #added this to try
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import pickle
import xgboost
from xgboost import XGBClassifier
import plotly as py
import plotly.graph_objs as go
import seaborn as sns

In [ ]:
df = pd.read_csv('../assets/Diamond Capstone Round Only.csv')
df.head()

## Exploratory Data Analysis

In [ ]:
df.shape
#>100K records
#18 features! more than the public dataset I worked with previously

In [ ]:
#check for missing data
df.isnull().sum()
#some missing data, but pretty small compared to the number of records (<1%) so I will drop them rather than trying to fill them in

In [ ]:
#drop the records that are missing color
print(df['color'].value_counts(dropna = False))
df.dropna(subset=['color'], inplace=True)
print(df['color'].value_counts(dropna=False))

In [ ]:
df.isnull().sum()

In [ ]:
#check what the possible values are for polish, symmetry, and flourescence

In [ ]:
print(df['fluorescence'].value_counts(dropna=False))

In [ ]:
print(df['symmetry'].value_counts(dropna=False))

In [ ]:
print(df['polish'].value_counts(dropna=False))

In [ ]:
#looks like the vast majority of the data for polish, symmetry and fluorescence are the same, so I can probably just set null values to 
#but since the missing data is small I'll just drop them

In [ ]:
df.dropna(subset=['fluorescence'], inplace=True)
df.dropna(subset=['symmetry'], inplace=True)
df.dropna(subset=['polish'], inplace=True)

In [ ]:
df.shape
#down to 110K records vs. original dataset of 111K (99%)

In [ ]:
#take a look at the datatypes
df.dtypes

In [ ]:
#double check that I only have round diamonds in this analysis
df['shape'].value_counts()

In [ ]:
#other observations from the data types:
#For linear regression, I will need to re-code cut, color, and clarity since they are non-numerical
#I will also re-code certificateLab
#For polish, symmetry and flourescence I will likely have to group values before re-coding since there are a lot of possible values
#I will not use certificateid since it is just an identifier
#I will likely not need measurements either since it probably correlated with other features

## Re-code Non-Numerical Features

In [ ]:
df['color'].value_counts()
#usually color only goes up to M? I will drop the others; not sure what OP, UV and XY are

In [ ]:
print(df['color'].value_counts())
df = df.drop(df[df["color"]=='N'].index)
df = df.drop(df[df["color"]=='OP'].index)
df = df.drop(df[df["color"]=='UV'].index)
df = df.drop(df[df["color"]=='XY'].index)
print(df['color'].value_counts())

In [ ]:
#re-code the remaining colors to numerical
print(df['color'].value_counts())
df['color'] = df['color'].map({"D":1, "E": 2, "F":3, "G":4, "H":5, "I":6, "J":7, "K":8, "L":9, "M":10})
print(df['color'].value_counts())

In [ ]:
df.shape

In [ ]:
df['clarity'].value_counts()
#one has a clarity of none but the rest look fine; drop that one record

In [ ]:
print(df['clarity'].value_counts())
df = df.drop(df[df["clarity"]=='None'].index)
print(df['clarity'].value_counts())

In [ ]:
#re-code the remaining clarity to numerical
print(df['clarity'].value_counts())
df['clarity'] = df['clarity'].map({"FL":1, "IF": 2, "VVS1":3, "VVS2":4, "VS1":5, "VS2":6, "SI1":7, "SI2":8, "I1":9})
print(df['clarity'].value_counts())
#based on the gia guide: https://4cs.gia.edu/en-us/diamond-clarity/

In [ ]:
df.shape

In [ ]:
df['cut'].value_counts()
#looks normal

In [ ]:
print(df['cut'].value_counts())
df['cut'] = df['cut'].map({"Ideal":1, "Excellent": 2, "Very Good":3, "Good":4, "Fair":5})
print(df['cut'].value_counts())

In [ ]:
df['certificateLab'].value_counts()
#vast majority are GIA
#the only other one I've seen is IGI
#group the others

In [ ]:
print(df['certificateLab'].value_counts())
df['certificateLab'] = df['certificateLab'].map({"GIA":"GIA", "IGI":"IGI", "HRD":"Other", "AGS":"Other", "B2C":"Other"})
print(df['certificateLab'].value_counts())
#probably not the most efficient way to rename these, but wanted to keep the logic consistent with the rest of the re-coding

In [ ]:
#now I will get dummies to turn these into numericals
ohe=pd.get_dummies(df['certificateLab'])
print(ohe.columns)
ohe.head()

In [ ]:
# append these new columns to the original dataset
df=pd.concat([df, ohe], axis=1) #append columns, not rows
df.shape
#3 extra columns now

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#remove the original column that we replaced with one-hot encoding
df=df.drop(['certificateLab'], axis=1).copy()
df.head()

In [ ]:
df.shape

In [ ]:
df['polish'].value_counts()
#similar to how cut is classified
#seeing some duplicates so will need to investigate and clean that up

In [ ]:
df[df["polish"]=='Excellent'].shape

In [ ]:
#used a quick pivot table in excel to find the issue; a lot of spaces at the end of the labels
df['polish'] = df['polish'].str.strip()

In [ ]:
df['polish'].value_counts()
#now it looks clean

In [ ]:
print(df['polish'].value_counts())
df['polish'] = df['polish'].map({"Ideal":1, "Excellent": 2, "Very Good":3, "Good":4, "Fair":5})
print(df['polish'].value_counts())

In [ ]:
df['symmetry'].value_counts()
#looks like the same issue as above

In [ ]:
df['symmetry'] = df['symmetry'].str.strip()

In [ ]:
df['symmetry'].value_counts()
#now it looks clean

In [ ]:
print(df['symmetry'].value_counts())
df['symmetry'] = df['symmetry'].map({"Ideal":1, "Excellent": 2, "Very Good":3, "Good":4, "Fair":5, "Poor":6})
print(df['symmetry'].value_counts())

In [ ]:
df['fluorescence'].value_counts()
#GIA only grades fluorescence as "None, Faint, Medium, Strong, Very Strong"; not sure how to group the other ones
#It also seems like this is not a very important factor for purchasers so I will exclude from my model for now

In [ ]:
df.shape

In [ ]:
df.dtypes

## Modeling the Price

In [ ]:
#now that the non-numerical data has been recoded, check correlations to see which features are most important
corrs = df.corr()
corrs

In [ ]:
#heatmap to see the correlations more clearly
plt.figure(figsize=(12,8))
sns.heatmap(corrs);

In [ ]:
corrs['price'].sort_values()
#similar to my last diamond price analysis; not seeing anything with strong correlation to price besides "carat". 

In [ ]:
#take a quick look ta the target variable
df.describe()
#the scale of price is weird?

In [ ]:
#make a copy of the data with only the features I will model
df_linear= df[['price', 'carat', 'cut', 'clarity', 'color', 'Other', 'IGI', 'GIA', 'polish', 'symmetry']].copy()
df_linear.head()

In [ ]:
#declare the target variable
y = df_linear['price']
y.shape

In [ ]:
#declare the model features
X = df_linear.drop(['price'], axis=1) #just exclude price
X.shape

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=12) #using the standard test size for now

In [ ]:
#instantiate the SKlearn algorithm
mymodel = LinearRegression(fit_intercept=True, 
                           normalize=False, 
                           copy_X=True, 
                           n_jobs=None, 
                           positive=False)

In [ ]:
#fit the model to the training dataset
mymodel.fit(X_train, y_train)

In [ ]:
print(mymodel)

In [ ]:
#what is the intercept?
mymodel.intercept_
#according to my model intercept, if carat, cut, clarity, color, polish, symmetry and the rating agencies were all 0, the diamond would be priced at -$6K
#doesn't really tell us much since that wouldn't happen in the real world

In [ ]:
#what is the equation for this mymodel?
cols=list(X.columns)
coefs=list(mymodel.coef_)
xcoefs = [(coefs[i], cols[i]) for i in range(0, len(cols))]
output = f'y = {round(mymodel.intercept_,2)} + {round(xcoefs[0][0],2)}*{xcoefs[0][1]} + {round(xcoefs[1][0],2)}*{xcoefs[1][1]} + {round(xcoefs[2][0],2)}*{xcoefs[2][1]} + {round(xcoefs[3][0],2)}*{xcoefs[3][1]} + {round(xcoefs[4][0],2)}*{xcoefs[4][1]} + {round(xcoefs[5][0],2)}*{xcoefs[5][1]} + {round(xcoefs[6][0],2)}*{xcoefs[6][1]}+ {round(xcoefs[7][0],2)}*{xcoefs[7][1]}+ {round(xcoefs[8][0],2)}*{xcoefs[8][1]}'
print("Regression Equation: ", output)
#cut, clarity, and color all have negative intercepts because I mapped the "best" to 1 with ascending values after that so that makes sense

In [ ]:
#for the graph below
coefs=list(mymodel.coef_)
cols=list(X.columns)

In [ ]:
#turn these into a dataframe
results = pd.DataFrame(list(zip(cols, coefs)), columns=['feature','coeff'])
results 

In [ ]:
#show results as a bar chart
import plotly.express as px
fig = px.bar(x=results['feature'], y=results['coeff'])
fig.update_layout(
    yaxis=dict( title='coefficient'),
    xaxis=dict( title='predictor'),
)
fig.show()
#surprisingly, cut doesn't seem to have much of an impact
#the magnitude of impact for clarity and color match my understanding based on the Rapaport pricing guide

In [ ]:
#predict the y-values on the testing dataset
y_preds = mymodel.predict(X_test)

In [ ]:
#compare some of the results of the predictions 
print([round(x,2) for x in y_preds[:5]])
print(list(y_test[:5]))
#looks very wrong...

In [ ]:
#compare predictions to known values
plt.figure(figsize=(6,6))
ax = sns.regplot(x = y_preds, 
                 y = y_test, 
                 scatter_kws={"color": "black"}, 
                 line_kws={"color": "red"})
ax.set(xlabel='predicted values', ylabel='true values');
#something weird is going on

## Model Evaluation

In [ ]:
#root mean sq error
rmse = sqrt(metrics.mean_squared_error(y_test, y_preds))
rmse

In [ ]:
#is that more or less than if we just used the average as our model?
avg_val = y_train.mean()
avg_val

In [ ]:
#what would the error be if I predicted the average price for all diamonds?
comparison = np.full((len(y_test), ), avg_val)
comparison[:10]

In [ ]:
#compare that to my predictions
y_preds[:10]
#something is very off because the values are negative

In [ ]:
#compare these two:
sqrt(metrics.mean_squared_error(y_test, comparison))

In [ ]:
# check R-2 (coefficient of determination)
r2 = metrics.r2_score(y_test, y_preds)
round(r2, 2)
#very bad...need to fix something

## Additional EDA / Cleanup

In [ ]:
df_linear['price'].describe()
#why is it like this??
#do I need to remove outliers?

In [ ]:
df_linear['color'].describe()
#this one looks normal

In [ ]:
#look at the spread of the data to see if there are outliers
plt.scatter(df_linear['carat'], df_linear['price']);
#there are definitely some outliers probably making it hard to predict well
#not sure how to systematically remove the outliers but I will cut it off at 10 carats for now since most of the data lies below that

In [ ]:
df_linear.shape

## Remove Outliers and Re-Model

In [ ]:
#drop the outliers
df_linear = df_linear[(df["carat"]<10)]

In [ ]:
df_linear.shape
#didn't lose much data

In [ ]:
df_linear.head()

In [ ]:
#try the model again
#declare the target variable
y = df_linear['price']
y.shape

In [ ]:
#declare the model features
X = df_linear.drop(['price'], axis=1) #just exclude price
X.shape

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=12) #using the standard test size for now

In [ ]:
#instantiate the SKlearn algorithm
mymodel = LinearRegression(fit_intercept=True, 
                           normalize=False, 
                           copy_X=True, 
                           n_jobs=None, 
                           positive=False)

In [ ]:
#fit the model to the training dataset
mymodel.fit(X_train, y_train)

In [ ]:
print(mymodel)

In [ ]:
#what is the intercept?
mymodel.intercept_
#according to my model intercept, if carat, cut, clarity, color, polish, symmetry and the rating agencies were all 0, the diamond would be priced at -$2K
#this looks better than before already!

In [ ]:
#what is the equation for this mymodel?
cols=list(X.columns)
coefs=list(mymodel.coef_)
xcoefs = [(coefs[i], cols[i]) for i in range(0, len(cols))]
output = f'y = {round(mymodel.intercept_,2)} + {round(xcoefs[0][0],2)}*{xcoefs[0][1]} + {round(xcoefs[1][0],2)}*{xcoefs[1][1]} + {round(xcoefs[2][0],2)}*{xcoefs[2][1]} + {round(xcoefs[3][0],2)}*{xcoefs[3][1]} + {round(xcoefs[4][0],2)}*{xcoefs[4][1]} + {round(xcoefs[5][0],2)}*{xcoefs[5][1]} + {round(xcoefs[6][0],2)}*{xcoefs[6][1]}+ {round(xcoefs[7][0],2)}*{xcoefs[7][1]}+ {round(xcoefs[8][0],2)}*{xcoefs[8][1]}'
print("Regression Equation: ", output)

In [ ]:
#for the graph below
coefs=list(mymodel.coef_)
cols=list(X.columns)

In [ ]:
#turn these into a dataframe
results = pd.DataFrame(list(zip(cols, coefs)), columns=['feature','coeff'])
results 

In [ ]:
#show results as a bar chart
import plotly.express as px
fig = px.bar(x=results['feature'], y=results['coeff'])
fig.update_layout(
    yaxis=dict( title='coefficient'),
    xaxis=dict( title='predictor'),
)
fig.show()

In [ ]:
#predict the y-values on the testing dataset
y_preds = mymodel.predict(X_test)

In [ ]:
#compare some of the results of the predictions 
print([round(x,2) for x in y_preds[:5]])
print(list(y_test[:5]))
#still looks wrong...

In [ ]:
#compare predictions to known values
plt.figure(figsize=(6,6))
ax = sns.regplot(x = y_preds, 
                 y = y_test, 
                 scatter_kws={"color": "black"}, 
                 line_kws={"color": "red"})
ax.set(xlabel='predicted values', ylabel='true values');
#still too much spread

## Model Evaluation v2

In [ ]:
# check R-2 (coefficient of determination)
r2 = metrics.r2_score(y_test, y_preds)
round(r2, 2)
#better but probably still need to make the cutoff lower due to high spread in the larger carat ranges

## Remove Outliers and Re-Model v2

In [ ]:
#drop the outliers
df_linear = df_linear[(df_linear["carat"]<2)]
#did a quick check for outliers using IQR in excel; seems like most of the data is below 2 carats which aligns with what I expected

In [ ]:
df_linear.shape
#still have over 90% of the original data

In [ ]:
#try the model again
#declare the target variable
y = df_linear['price']
y.shape

In [ ]:
#declare the model features
X = df_linear.drop(['price'], axis=1) #just exclude price
X.shape

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=12) #using the standard test size for now

In [ ]:
#instantiate the SKlearn algorithm
mymodel = LinearRegression(fit_intercept=True, 
                           normalize=False, 
                           copy_X=True, 
                           n_jobs=None, 
                           positive=False)

In [ ]:
#fit the model to the training dataset
mymodel.fit(X_train, y_train)

In [ ]:
print(mymodel)

In [ ]:
#what is the intercept?
mymodel.intercept_
#according to my model intercept, if carat, cut, clarity, color, polish, symmetry and the rating agencies were all 0, the diamond would be priced at -$200
#this looks better than before already!

In [ ]:
#what is the equation for this mymodel?
cols=list(X.columns)
coefs=list(mymodel.coef_)
xcoefs = [(coefs[i], cols[i]) for i in range(0, len(cols))]
output = f'y = {round(mymodel.intercept_,2)} + {round(xcoefs[0][0],2)}*{xcoefs[0][1]} + {round(xcoefs[1][0],2)}*{xcoefs[1][1]} + {round(xcoefs[2][0],2)}*{xcoefs[2][1]} + {round(xcoefs[3][0],2)}*{xcoefs[3][1]} + {round(xcoefs[4][0],2)}*{xcoefs[4][1]} + {round(xcoefs[5][0],2)}*{xcoefs[5][1]} + {round(xcoefs[6][0],2)}*{xcoefs[6][1]}+ {round(xcoefs[7][0],2)}*{xcoefs[7][1]}+ {round(xcoefs[8][0],2)}*{xcoefs[8][1]}'
print("Regression Equation: ", output)

In [ ]:
#for the graph below
coefs=list(mymodel.coef_)
cols=list(X.columns)

In [ ]:
#turn these into a dataframe
results = pd.DataFrame(list(zip(cols, coefs)), columns=['feature','coeff'])
results 

In [ ]:
#show results as a bar chart
import plotly.express as px
fig = px.bar(x=results['feature'], y=results['coeff'])
fig.update_layout(
    yaxis=dict( title='coefficient'),
    xaxis=dict( title='predictor'),
)
fig.show()

In [ ]:
#predict the y-values on the testing dataset
y_preds = mymodel.predict(X_test)

In [ ]:
#compare some of the results of the predictions 
print([round(x,2) for x in y_preds[:5]])
print(list(y_test[:5]))
#much closer!!!

In [ ]:
#compare predictions to known values
plt.figure(figsize=(6,6))
ax = sns.regplot(x = y_preds, 
                 y = y_test, 
                 scatter_kws={"color": "black"}, 
                 line_kws={"color": "red"})
ax.set(xlabel='predicted values', ylabel='true values');
#still a lot more spread than I expected

In [ ]:
# check R-2 (coefficient of determination)
r2 = metrics.r2_score(y_test, y_preds)
round(r2, 2)
#much better than before but still an interesting curve; seems like for the lower carat ranges, the linear relationship / modeling works well, but in the higher carat ranges, prices diverge more and start to look more parabolic. this makes sense because these sizes are probably much rarer so buyers will have to pay an extra premium above the features I have included in the model

In [ ]:
#other things I want to try:
#is there a parabolic model?
#or I could split the model even further into different carat ranges...
#have to check if there is enough data for that though

In [ ]:
#add the scatter plots by carat range here

## Try Random Forest Regressor

In [ ]:
#just run it on the original df + EDA

In [ ]:
df.shape #this is before I removed any of the data >2 carats #now I need to try it with less data...

In [ ]:
df_rf= df[['price', 'carat', 'cut', 'clarity', 'color', 'Other', 'IGI', 'GIA', 'polish', 'symmetry']].copy()
df_rf.head()

In [ ]:
df_rf.shape

In [ ]:
df_rf.isnull().sum()
#confirming no missing data

In [ ]:
df_rf.dtypes
#confirming all are non-categorial

In [ ]:
y = df_rf['price']
print(y.shape)

In [ ]:
#remove price from features list
X = df_rf.drop('price', axis=1)
print(X.shape)

In [ ]:
#train, test, split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
#try standardizing the features
scaler = StandardScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#instantiate the model; used random paramaters for now
rand_forest = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth = 5, criterion = 'squared_error', random_state = 42)

In [ ]:
rand_forest.fit(X_train_scaled, y_train)

In [ ]:
#Predict
y_preds=rand_forest.predict(X_test_scaled)
print('preds', list(y_preds[:10]))
print('truth', list(y_test[:10]))
#doesn't look too good

In [ ]:
#evaluate the model
prediction = rand_forest.predict(X_train_scaled)
mse = mean_squared_error(y_test, y_preds)
rmse = mse**.5
print(mse)
print(rmse)
#the error looks high..

## Try to Optimize with Grid Search

In [ ]:
#define grid
grid = { 
    'n_estimators': [100,200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [3,4,5,6,7],
    'random_state' : [42]
}
#didn't try this one again after scaling since scaling didn't improve the error

In [ ]:
#comment out for now so it doesn't run every time
#CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5)
#CV_rfr.fit(X_train, y_train)

In [ ]:
#print(CV_rfr.best_params_)

In [ ]:
#try again with these parameters
rand_forest = RandomForestRegressor(n_estimators = 100, max_features = 'sqrt', max_depth = 7, random_state = 42)

In [ ]:
rand_forest.fit(X_train, y_train)

In [ ]:
#Predict
y_preds=rand_forest.predict(X_test)
print('preds', list(y_preds[:10]))
print('truth', list(y_test[:10]))
#still doesn't look great

In [ ]:
#evaluate the model
prediction = rand_forest.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = mse**.5
print(mse)
print(rmse)
#the error is lower but still high

## Try XG Boost Regressor

In [ ]:
#decide which features I want to include and the cutoff for carat size; no cutoff for now
df_xg= df[['price', 'carat', 'cut', 'clarity', 'color', 'Other', 'IGI', 'GIA', 'polish', 'symmetry']].copy()
df_xg.head()

In [ ]:
df_xg.shape

In [ ]:
df_xg.isnull().sum()
#confirming no missing data

In [ ]:
df_xg.dtypes
#confirming all are non-categorial

In [ ]:
y = df_xg['price']
print(y.shape)

In [ ]:
#remove price from features list
X = df_xg.drop('price', axis=1)
print(X.shape)

In [ ]:
#train, test, split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
#set up the model (arbitrary parameters for now)

xgb_model = XGBRegressor(objective = 'reg:linear', n_estimators = 10, seed = 123)

In [ ]:
#fit the model
xgb_model.fit(X_train, y_train)

In [ ]:
#predict
y_preds = xgb_model.predict(X_test)

In [ ]:
#evaluate with RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print("RMSE : % f" %(rmse))

In [ ]:
#definitely better than random forest, but probably a lot better if I cut the data into <2 carats too
#can also try grid search after I cut the data

## XG Boost Part 2 (<=2 Carats Only)

In [ ]:
#create new dataframe with only diamonds 2 carats and below for use in any model
df_2ct = df[(df["carat"]<2)]

In [ ]:
df_2ct.shape

In [ ]:
#decide which features I want to include
df_xg2= df_2ct[['price', 'carat', 'cut', 'clarity', 'color', 'Other', 'IGI', 'GIA', 'polish', 'symmetry']].copy()
df_xg2.head()

In [ ]:
df_xg2.shape

In [ ]:
df_xg2.isnull().sum()

In [ ]:
df_xg2.dtypes

In [ ]:
y = df_xg2['price']
print(y.shape)

In [ ]:
#remove price from features list
X = df_xg2.drop('price', axis=1)
print(X.shape)

In [ ]:
#train, test, split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
#set up the model (arbitrary parameters for now)

xgb_model = XGBRegressor(objective = 'reg:linear', n_estimators = 10, seed = 123)

In [ ]:
#fit the model
xgb_model.fit(X_train, y_train)

In [ ]:
#predict
y_preds = xgb_model.predict(X_test)

In [ ]:
#evaluate with RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print("RMSE : % f" %(rmse))
#much better!!!
#does this mean that on average my model is "wrong" by $600?

In [ ]:
#plot the values
plt.figure(figsize=(6,6))
ax = sns.regplot(x = y_preds, 
                 y = y_test, 
                 scatter_kws={"color": "black"}, 
                 line_kws={"color": "red"})
ax.set(xlabel='predicted values', ylabel='true values');
#this one looks so much better

In [ ]:
#remaining question for Sonyah - how else do I evaluate the error / accuracy of this model?

## Try to Optimize XG Boost with Grid Search

In [ ]:
#got this from some googling - not sure what all of it means...
#comment out for now
#xgb_gs = XGBRegressor()
#parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#              'objective':['reg:linear'],
#              'learning_rate': [.03, 0.05, .07], #so called `eta` value
#              'max_depth': [5, 6, 7],
#              'min_child_weight': [4],
#              'silent': [1],
#              'subsample': [0.7],
#              'colsample_bytree': [0.7],
#              'n_estimators': [500]}

In [ ]:
#xgb_grid = GridSearchCV(xgb_gs,
#                        parameters,
#                        cv = 2,
#                        n_jobs = 5,
#                        verbose=True)

In [ ]:
#xgb_grid.fit(X_train,
#        y_train)

In [ ]:
#print(xgb_grid.best_score_)
#print(xgb_grid.best_params_)

In [ ]:
#maybe add a summary of the different evaluation metrics from the different models

In [ ]:
#new model with grid search parameters
xgb_model = XGBRegressor('colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7)

In [ ]:
#fit the model
xgb_model.fit(X_train, y_train)

In [ ]:
#predict
y_preds = xgb_model.predict(X_test)

In [ ]:
#evaluate with RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print("RMSE : % f" %(rmse))
#it's still the same RMSE without the grid search?

In [ ]:
#how to interpret this though?

In [ ]:
y_train

In [ ]:
#is this error more or less than if we just used the average as our model?
avg_val = y_train.mean()
avg_val

In [ ]:
#what would the error be if I predicted the average price for all diamonds?
comparison = np.full((len(y_test), ), avg_val)
comparison[:10]

In [ ]:
#compare that to my predictions
y_preds[:10]
#something is very off because the values are negative

In [ ]:
#compare these two:
sqrt(metrics.mean_squared_error(y_test, comparison))

In [ ]:
# check R-2 (coefficient of determination)
r2 = metrics.r2_score(y_test, y_preds)
round(r2, 2)
#very bad...need to fix something